In [6]:
import keras
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import Layer
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Lambda, ReLU, Add,Dropout, Activation, Flatten, Input, PReLU,SeparableConv2D, Conv2DTranspose,concatenate,Convolution2D,ZeroPadding2D,Add,MaxPool2D
from tensorflow.keras.layers import Conv2D,Conv2DTranspose, Activation,MaxPooling3D, MaxPooling2D, BatchNormalization, UpSampling2D,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ReduceLROnPlateau,LearningRateScheduler, TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda
from tensorflow.keras.optimizers import Adam
import random
from tensorflow.keras.layers import Layer, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.layers import Dense, Reshape, Conv2D, Multiply, Concatenate, Add
import keras.ops as ops




class ChannelShuffle(Layer):
    def __init__(self, groups=2, **kwargs):
        super(ChannelShuffle, self).__init__(**kwargs)
        self.groups = groups

    def call(self, x):
        # x: (batch, h, w, c)
        batch_size, h, w, c = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2], tf.shape(x)[3]
        channels_per_group = c // self.groups
        x = tf.reshape(x, [batch_size, h, w, self.groups, channels_per_group])
        x = tf.transpose(x, [0, 1, 2, 4, 3])
        x = tf.reshape(x, [batch_size, h, w, c])
        return x



def two_path(input_tensor, filters, kernel_size, strides=(1, 1), padding='valid'):



        filters_per_group = filters // 2

        input_tensor_shuffled = ChannelShuffle(groups=2)(input_tensor)

        group1, group2 = ops.split(input_tensor_shuffled, 2, axis=-1)


        # H path- First stage of convolution
        convH1 = tf.keras.layers.Conv2D(filters=filters_per_group,
                                          kernel_size=kernel_size,
                                          kernel_initializer='he_uniform',
                                          strides=strides,
                                          padding=padding)(group1)
        convH1= BatchNormalization()(convH1)
        convH1= ReLU()(convH1)



        # L path- First stage of convolution
        convL1 = tf.keras.layers.Conv2D(filters=filters_per_group,
                                          kernel_size=kernel_size,
                                          kernel_initializer='he_uniform',
                                          dilation_rate= 2,
                                          strides=strides,
                                          padding=padding)(group2)
        convL1    = BatchNormalization()(convL1)
        convL1    = ReLU()(convL1)



        # Concat first stage
        X1     = Concatenate(axis=-1)([convH1,convL1])


        # H path- Second stage of convolution
        convH2 = tf.keras.layers.SeparableConv2D(filters=filters_per_group,
                                          kernel_size=kernel_size,
                                          depthwise_initializer='he_uniform',
                                          pointwise_initializer='he_uniform',
                                          strides=strides,
                                          padding=padding)(X1)
        convH2= BatchNormalization()(convH2)
        convH2= ReLU()(convH2)



        # L path- Second stage of convolution
        convL2 = tf.keras.layers.SeparableConv2D(filters=filters_per_group,
                                          kernel_size=kernel_size,
                                          depthwise_initializer='he_uniform',
                                          pointwise_initializer='he_uniform',
                                          dilation_rate= 2,
                                          strides=strides,
                                          padding=padding)(X1)
        convL2= BatchNormalization()(convL2)
        convL2= ReLU()(convL2)


        # Concat second stage
        X2     = Concatenate(axis=-1)([convH2,convL2])



        # H-path-Third stage of convolution
        convH3 = tf.keras.layers.Conv2D(filters=filters_per_group,
                                          kernel_size=kernel_size,
                                          kernel_initializer='he_uniform',
                                          strides=strides,
                                          padding=padding)(X2)
        convH3= BatchNormalization()(convH3)
        convH3= ReLU()(convH3)


        # L-path-Third stage of convolution
        convL3 = tf.keras.layers.Conv2D(filters=filters_per_group,
                                          kernel_size=kernel_size,
                                          kernel_initializer='he_uniform',
                                          dilation_rate= 2,
                                          strides=strides,
                                          padding=padding)(X2)
        convL3= BatchNormalization()(convL3)
        convL3= ReLU()(convL3)


        # Final concat
        output_tensor = Concatenate(axis=-1)([convH3,convL3])

        return output_tensor

#********************************
#********************************



input = tf.keras.Input(shape=(64, 64, 1))

# Block 1

b1= tf.keras.layers.Conv2D(filters=66, kernel_size=(3, 3), kernel_initializer='he_uniform', padding='same')(input)
b1 = BatchNormalization()(b1)
b1 = ReLU()(b1)
b1 = tf.keras.layers.SeparableConv2D(filters=66, kernel_size=(3, 3), depthwise_initializer='he_uniform', pointwise_initializer='he_uniform', padding='same')(b1)
b1 = BatchNormalization()(b1)
b1 = ReLU()(b1)
b1 = tf.keras.layers.Conv2D(filters=66, kernel_size=(3, 3), kernel_initializer='he_uniform', padding='same')(b1)
b1 = BatchNormalization()(b1)
b1 = MaxPooling2D(pool_size=2)(b1)
b1 = ReLU()(b1)
b1 = Dropout(0.4)(b1)

# Block 2

b2i = two_path(b1, filters=72, kernel_size=3, strides=(1, 1), padding='same')
b2 = MassAtt(b2i, ratio=4)
b2 = b2 * b2i
b2 = Conv2D(72, kernel_size=(1, 1),kernel_initializer='he_uniform', padding='same')(b2)
b2 = BatchNormalization()(b2)
b2 = MaxPooling2D(pool_size=2)(b2)
b2 = ReLU()(b2)
b2 = Dropout(0.4)(b2)

# Block 3

b3 = tf.keras.layers.Conv2D(filters=78, kernel_size=(3, 3), kernel_initializer='he_uniform', padding='same')(b2)
b3 = BatchNormalization()(b3)
b3 = ReLU()(b3)
b3 = tf.keras.layers.SeparableConv2D(filters=78, kernel_size=(3, 3), depthwise_initializer='he_uniform', pointwise_initializer='he_uniform', padding='same')(b3)
b3 = BatchNormalization()(b3)
b3 = ReLU()(b3)
b3 = tf.keras.layers.Conv2D(filters=78, kernel_size=(3, 3), kernel_initializer='he_uniform', padding='same')(b3)
b3 = BatchNormalization()(b3)
b3 = MaxPooling2D(pool_size=2)(b3)
b3= ReLU()(b3)
b3= Dropout(0.4)(b3)

# Block 4

b4i = two_path(b3, filters=84, kernel_size=3, strides=(1, 1), padding='same')
b4 = MassAtt(b4i, ratio=4)
b4 = b4 * b4i
b4 = Conv2D(84, kernel_size=(1, 1),kernel_initializer='he_uniform', padding='same')(b4)
b4 = BatchNormalization()(b4)
b4 = MaxPooling2D(pool_size=2)(b4)
b4 = ReLU()(b4)
b4 = Dropout(0.4)(b4)

b1 = PWFS()(b1)
b2 = PWFS()(b2)
b3 = PWFS()(b3)

b1 = GlobalAveragePooling2D()(b1)
b2 = GlobalAveragePooling2D()(b2)
b3 = GlobalAveragePooling2D()(b3)
b4 = GlobalAveragePooling2D()(b4)

f = Concatenate(axis=-1)([b1,b2,b3,b4])

output= Dense(8, activation='softmax')(f)

model = tf.keras.Model(inputs=input, outputs=output)

model.summary()





# Compile the model
model.compile(loss=CategoricalCrossentropy,
                optimizer=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
                metrics=['accuracy'])


learning_rate_reducer = ReduceLROnPlateau('val_loss', factor=0.1, patience=9, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir='./logs')
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=14, verbose=1, mode='auto')

# Fit the model on the current fold
model.fit(X_train, Y_train,
            batch_size=32,
            epochs=100,
            verbose=1,
            validation_data=(x_val_fold, y_val_fold),
            callbacks=[learning_rate_reducer, tensorboard, early_stopper])


test_loss, test_acc = model.evaluate(np.array(x_val_fold), np.array(y_val_fold), batch_size=32)
print('test_pub_acc:', test_acc, 'test_loss', test_loss)



Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 64, 64, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_74 (Conv2D)  │ (None, 64, 64,    │        660 │ input_layer_5[0]… │
│                     │ 66)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        264 │ conv2d_74[0][0]   │
│ (BatchNormalizatio… │ 66)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_83 (ReLU)     │ (None, 64, 64,    │          0 │ batch_normalizat… │
│                     │ 66)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_25 │ (None, 64, 64,    │      5,016 │ re_lu_83[0][0]    │
│ (SeparableConv2D)   │ 66)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        264 │ separable_conv2d… │
│ (BatchNormalizatio… │ 66)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_84 (ReLU)     │ (None, 64, 64,    │          0 │ batch_normalizat… │
│                     │ 66)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_75 (Conv2D)  │ (None, 64, 64,    │     39,270 │ re_lu_84[0][0]    │
│                     │ 66)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        264 │ conv2d_75[0][0]   │
│ (BatchNormalizatio… │ 66)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_17    │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 66)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_85 (ReLU)     │ (None, 32, 32,    │          0 │ max_pooling2d_17… │
│                     │ 66)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_17          │ (None, 32, 32,    │          0 │ re_lu_85[0][0]    │
│ (Dropout)           │ 66)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ channel_shuffle_9   │ (None, 32, 32,    │          0 │ dropout_17[0][0]  │
│ (ChannelShuffle)    │ 66)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ split_8 (Split)     │ [(None, 32, 32,   │          0 │ channel_shuffle_… │
│                     │ 33), (None, 32,   │            │                   │
│                     │ 32, 33)]          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_76 (Conv2D)  │ (None, 32, 32,    │     10,728 │ split_8[0][0]     │
│                     │ 36)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_77 (Conv2D)  │ (None, 32, 32,    │     10,728 │ split_8[0][1]     │
│                     │ 36)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 358,563 (1.37 MB)

 Trainable params: 356,451 (1.36 MB)

 Non-trainable params: 2,112 (8.25 KB)

NameError: name 'X_train' is not defined